In [1]:
#Ensures that the latest version of semantic-link libraries are loaded
%pip install -U semantic-link

%pip install -U semantic-link-labs

StatementMeta(, 4596065f-b686-4c04-b896-e4634227f54c, 9, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.7/341.7 kB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 135.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 179.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
###########################################################################################################
### Set Parameters
###########################################################################################################
#Define lakehouse and schema names
workspace_name = "amaser_dp700" #this is the workspace name where the notebook runs, and where the lakehouse lives
lakehouse_name = "Fit_Tool" #this is the name of the lakehouse, in the event that it needs to becreated by this notebook
schema_name = "fitschema"  #This is the name of the schema.  if using non-schema-enabled lakehouse, this MUST be dbo
semantic_model_name = "FabricInventory" 

#keyvault secret names: These need to be the names of the secrets in the given keyvault
key_vault_uri="https://fittool.vault.azure.net/"
tenant_id = "fit-tenant"
client_id = "fit-client"
client_secret = "fit-secret"

StatementMeta(, 4596065f-b686-4c04-b896-e4634227f54c, 11, Finished, Available, Finished)

In [3]:
#import necessary libraries
import sempy.fabric as fabric
#from sempy.fabric import set_service_principal
import pandas as pd
from datetime import date,datetime
import time
import sempy_labs as labs
from sempy_labs import admin, service_principal_authentication


# set default values used in future steps
full_lakehouse_name = f'{workspace_name}.{lakehouse_name}.{schema_name}'
current_date = date.today()
workspace_table_name = f'{full_lakehouse_name}.workspaces'
items_table_name = f'{full_lakehouse_name}.fabricitems'
capacities_table_name = f'{full_lakehouse_name}.capacities'

StatementMeta(, 4596065f-b686-4c04-b896-e4634227f54c, 12, Finished, Available, Finished)

In [4]:
## This cell gets all the workspaces, and loads them into the workspaces table

print("Starting Workspace table load....")

with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri,
    key_vault_tenant_id=tenant_id,
    key_vault_client_id=client_id,
    key_vault_client_secret=client_secret):
    all_workspaces_df = admin.list_workspaces()
    #display(all_workspaces_df)
    

#match names to table names for simplicity sake
workspaces_df = all_workspaces_df[['Name','Id','State','domainId','Capacity Id','Type']]
workspaces_df = workspaces_df.rename(columns={'State':'state'})
workspaces_df = workspaces_df.rename(columns={'Name':'workspace_name'})
workspaces_df = workspaces_df.rename(columns={'Id':'workspace_id'})
workspaces_df = workspaces_df.rename(columns={'domainId':'domain_id'})
workspaces_df = workspaces_df.rename(columns={'Capacity Id':'capacity_id'})
workspaces_df = workspaces_df.rename(columns={'Type':'type'})
workspaces_df['type'] = workspaces_df['type'].str.replace('Workspace','Shared')
#add the timestamp column to the dataframe
workspaces_df['timeval'] = current_date
#display(workspaces_df)

#delete the current date entries, so only one entry per day should exist
delete_workspace_qeuery = f"DELETE FROM {workspace_table_name} WHERE timeval = '{current_date}'"
print("Deleting workspace entries for the current date....")

spark.sql(delete_workspace_qeuery)
print("current date Workspace entries deleted.")


#insert rows into workspace table
spark_df = spark.createDataFrame(workspaces_df)
spark_df.write.mode("append").format("delta").saveAsTable(workspace_table_name)

print("Workspaces inserted")


StatementMeta(, 4596065f-b686-4c04-b896-e4634227f54c, 13, Finished, Available, Finished)

Starting Workspace table load....
Deleting workspace entries for the current date....
current date Workspace entries deleted.
Workspaces inserted


In [5]:
#get workspace items
print("Starting fabricItems....")

print("Deleting current date fabric item entries....")

del_items_query = f"DELETE FROM {items_table_name} WHERE timeval = '{current_date}'"
spark.sql(del_items_query)
print("current date item entries deleted.")



with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri,
    key_vault_tenant_id=tenant_id,
    key_vault_client_id=client_id,
    key_vault_client_secret=client_secret):

        items_df = admin.list_items()

#rename and clean up dataframe
items_df = items_df.rename(columns={'Capacity Id':'capacity_id'})
items_df = items_df.rename(columns={'Workspace Id':'workspace_id'})
items_df = items_df.rename(columns={'Creator Principal Type':'principal_type'})
items_df = items_df.rename(columns={'Creator Principal Display Name':'principal_name'})
items_df = items_df.rename(columns={'Last Updated Date':'last_updated'})
items_df = items_df.rename(columns={'State':'state'})
items_df = items_df.rename(columns={'Type':'item_type'})
items_df = items_df.rename(columns={'Item Name':'item_name'})
items_df = items_df.rename(columns={'Item Id':'item_id'})
items_df['last_updated'] = items_df['last_updated'].str.replace('T',' ')
del items_df['Creator User Principal Name']
del items_df['Creator Principal Id']
items_df['timeval'] = current_date
#display(items_df)
new_order = ["timeval", "item_name", "item_id","Description","item_type","workspace_id","capacity_id","principal_type","principal_name","last_updated","state"]

#reorder columns
final_items_df = items_df[new_order]
#display(final_items_df)
spark_df = spark.createDataFrame(final_items_df)

#display(spark_df)
#TEMPORARY TEST
#items_table_name = f'{full_lakehouse_name}.fabricitemstest'
spark_df.write.mode("append").format("delta").saveAsTable(items_table_name)
print(f'fabric items entered')


StatementMeta(, 4596065f-b686-4c04-b896-e4634227f54c, 14, Finished, Available, Finished)

Starting fabricItems....
Deleting current date fabric item entries....
current date item entries deleted.
fabric items entered


In [6]:
#get capacities items
print("Starting capacities....")


print("Deleting current date capacity entries....")

del_items_query = f"DELETE FROM {capacities_table_name} WHERE timeval = '{current_date}'"
spark.sql(del_items_query)
print("current date capacity entries deleted.")



with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri,
    key_vault_tenant_id=tenant_id,
    key_vault_client_id=client_id,
    key_vault_client_secret=client_secret):

        capacities_df = admin.list_capacities()

#match names to table
capacities_df = capacities_df.rename(columns={'Capacity Id':'capacity_id'})
capacities_df = capacities_df.rename(columns={'Capacity Name':'capacity_name'})
capacities_df = capacities_df.rename(columns={'Sku':'sku'})
capacities_df = capacities_df.rename(columns={'Region':'region'})
capacities_df = capacities_df.rename(columns={'State':'state'})
capacities_df = capacities_df.rename(columns={'Admins':'admins'})
capacities_df['timeval'] = current_date
#capacities_df['admins'] = capacities_df['admins'].str.replace('[]','')
del capacities_df['Users']
del capacities_df['admins']
#display(capacities_df)

spark_df = spark.createDataFrame(capacities_df)


spark_df.write.mode("append").format("delta").saveAsTable(capacities_table_name)
print(f'Capacities entered')


StatementMeta(, 4596065f-b686-4c04-b896-e4634227f54c, 15, Finished, Available, Finished)

Starting capacities....
Deleting current date capacity entries....
current date capacity entries deleted.
Capacities entered


In [7]:

#Create date table
current_year = datetime.now().year
end_year = current_year + 1
#print(end_year)
def create_date_table(start_date='2018-01-01', end_date=f'{end_year}-12-31'):
    """
    Creates a Pandas DataFrame representing a full date table.

    Args:
        start_date (str): The start date for the date table (format 'YYYY-MM-DD').
        end_date (str): The end date for the date table (format 'YYYY-MM-DD').

    Returns:
        pd.DataFrame: A DataFrame containing various date-related columns.
    """
    df = pd.DataFrame({"Date": pd.date_range(start_date, end_date)})

    df["Day"] = df.Date.dt.day_name()
    df["DayOfWeek"] = df.Date.dt.dayofweek + 1 # Monday=1, Sunday=7
    df["DayOfMonth"] = df.Date.dt.day
    df["DayOfYear"] = df.Date.dt.dayofyear

    df["Week"] = df.Date.dt.isocalendar().week.astype(int)
    df["WeekOfYear"] = df.Date.dt.isocalendar().week.astype(int) # Same as Week

    df["Month"] = df.Date.dt.month
    df["MonthName"] = df.Date.dt.month_name()
    df["Quarter"] = df.Date.dt.quarter
    df["Year"] = df.Date.dt.year
    df["YearQuarter"] = df.Date.dt.strftime('%Y-Q%q')
    df["Year-Month"] = df.Date.dt.strftime('%Y-%m')

    df["IsWeekend"] = df.Date.dt.dayofweek.isin([5, 6]) # Saturday=5, Sunday=6
    df["IsLeapYear"] = df.Date.dt.is_leap_year

    return df

spark.sql(f"DROP TABLE IF EXISTS {workspace_name}.{lakehouse_name}.{schema_name}.DimDate")
# Create the date table
date_table_df = create_date_table()
print("DimDate table dropped")

spark_df = spark.createDataFrame(date_table_df)
spark_df.write.mode("overwrite").format("delta").saveAsTable(f"{full_lakehouse_name}.DimDate")
print("DimDate table created")


StatementMeta(, 4596065f-b686-4c04-b896-e4634227f54c, 16, Finished, Available, Finished)

DimDate table dropped
DimDate table created


In [8]:

time.sleep(30)
print("Refreshing lakehouse data")

with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri,
    key_vault_tenant_id=tenant_id,
    key_vault_client_id=client_id,
    key_vault_client_secret=client_secret):

    labs.refresh_sql_endpoint_metadata(workspace=workspace_name,item=lakehouse_name,type='Lakehouse')

time.sleep(75)
with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri,
    key_vault_tenant_id=tenant_id,
    key_vault_client_id=client_id,
    key_vault_client_secret=client_secret):
    
    fabric.semantic_model.refresh_semantic_model(workspace=workspace_name, dataset=semantic_model_name)


print("Dataset refresh completed")


StatementMeta(, 4596065f-b686-4c04-b896-e4634227f54c, 17, Finished, Available, Finished)

Refreshing lakehouse data
🟢 The metadata of the SQL endpoint for the 'Fit_Tool' lakehouse within the 'amaser_dp700' workspace has been refreshed for all tables.
Dataset refresh completed
